### Fine-Tuning LLaMA-3.2 3B Instruct

This code will fine-tune the `LLaMA-3.2-3B-Instruct` model on the CounselChat dataset

In [1]:
import os

In [2]:
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [3]:
import torch
import pickle
from unsloth import FastLanguageModel, is_bfloat16_supported, train_on_responses_only
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from datasets import Dataset, DatasetDict
from trl import SFTTrainer

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


/data/mn27889/miniconda3/envs/mental-health-agents/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🦥 Unsloth Zoo will now patch everything to make training faster!


Some links to be used for fine-tuning
1. https://www.kdnuggets.com/fine-tuning-llama-using-unsloth
2. https://www.analyticsvidhya.com/blog/2024/12/fine-tuning-llama-3-2-3b-for-rag/
3. https://www.linkedin.com/pulse/step-guide-use-fine-tune-llama-32-dr-oualid-soula-xmnff/
4. https://medium.com/@alexandros_chariton/how-to-fine-tune-llama-3-2-instruct-on-your-own-data-a-detailed-guide-e5f522f397d7 (check the data format since it uses generation_token at end of input data since that might not be needed)
5. https://drlee.io/step-by-step-guide-fine-tuning-metas-llama-3-2-1b-model-f1262eda36c8
6. https://huggingface.co/blog/ImranzamanML/fine-tuning-1b-llama-32-a-comprehensive-article
7. https://blog.futuresmart.ai/fine-tune-llama-32-vision-language-model-on-custom-datasets
8. https://github.com/meta-llama/llama-cookbook/blob/main/getting-started/finetuning/multigpu_finetuning.md



### Reading the Counsel Chat Dataset

In [4]:
with open('processed_data/counselchat_top_votes_train.pkl', 'rb') as file:
    dataset_top_votes_train = pickle.load(file)

dataset_top_votes_train.head()

,topic,question,answerText
0,relationships,I am currently suffering from erectile dysfunc...,"Hi, First and foremost, I want to acknowledge ..."
1,family-conflict,For the past week or so me and my boyfriend ha...,Forgetting one's emotions is impossible.Since ...
2,depression,I am in high school and have been facing anxie...,"Hi Helena,I felt a bit sad when I read this. T..."
3,anxiety,I'm concerned about my boyfriend. I suffer fro...,Hello! Thank you for your question. There are ...
4,spirituality,"I'm a Christian teenage girl, and I have lost ...",Having sex with your boyfriend is and was a mi...


In [5]:
with open('processed_data/counselchat_top_votes_test.pkl', 'rb') as file:
    dataset_top_votes_test = pickle.load(file)

dataset_top_votes_test.head()

,topic,question,answerText
0,relationships,I had to go to the emergency room today to get...,It is extremely frustrating when our significa...
1,marriage,What makes a healthy marriage last? What makes...,"This is a fantastic question. In one sentence,..."
2,relationships,"I'm a female freshman in high school, and this...","First off, I think it is great that you are wi..."
3,intimacy,"My wife and I are newly married, about 2 month...","You are newly married, you Have a hectic sched..."
4,legal-regulatory,"I think I have depression, anxiety, bipolar di...",It can be difficult to get counseling if you d...


Creating the HuggingFace dataset using Pandas Dataframe

In [6]:
dataset_train = Dataset.from_pandas(dataset_top_votes_train)
dataset_test = Dataset.from_pandas(dataset_top_votes_test)

In [7]:
dataset = DatasetDict()
dataset['train'] = dataset_train
dataset['test'] = dataset_test

In [8]:
dataset

DatasetDict({
    train: Dataset({
        features: ['topic', 'question', 'answerText'],
        num_rows: 690
    })
    test: Dataset({
        features: ['topic', 'question', 'answerText'],
        num_rows: 173
    })
})

### Fine-Tuning Code

#### Loading the model and tokenizer

In [ ]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Llama-3.2-3B-Instruct",
    max_seq_length = 2048,
    load_in_4bit = False, # 4 bit quantization to reduce memory
    load_in_8bit = True, # [NEW!] A bit more accurate, uses 2x memory
    full_finetuning = False, # [NEW!] We have full finetuning now!
    dtype=None, #None for auto-detection. Can be torch.bfloat16 or torch.float16 (will be automatically detected)
    device_map="auto"
)

#### Setting up the PEFT settings for the model

https://huggingface.co/blog/damjan-k/rslora\
https://medium.com/@fartypantsham/what-rank-r-and-alpha-to-use-in-lora-in-llm-1b4f025fd133

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 64, #max_full_rank=64 by default in FastLanguageModel
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 64, #scaling_factor = lora_alpha/r. If we select lora_alpha = 2 * r then it will multiply the adapter weights by 2 which can be un-ncessary
    lora_dropout = 0.1,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    use_rslora = True,
    loftq_config = None,
)

#### Forming the chat template

In [ ]:
# Define a function to apply the chat template
def format_chat_template(example):
        
    messages = [
        {"role": "system", "content": "You are an expert mental health professional trained to counsel and guide patients suffering from ill mental-health"},
        {"role": "user", "content": example['question']},
        {"role": "assistant", "content": example['answerText']}
    ]
    
    prompt = tokenizer.apply_chat_template(messages, tokenize=False)

    return {"text": prompt}

In [ ]:
dataset_formatted = dataset.map(format_chat_template)

In [ ]:
print(dataset_formatted['train']['text'][0])

#### Initializing the TRL SFTTrainer and related Arguments

In [ ]:
# full_model_path = "./llama32-sft-full-counselchat" #use for full finetuning
peft_model_path = "./llama32-sft-peft-counselchat" #use for LoRA based fine-tuning

training_args = TrainingArguments(
        output_dir=peft_model_path,
        per_device_train_batch_size=4,
        per_device_eval_batch_size=2,
        # gradient_accumulation_steps=4,
        eval_strategy="steps",
        eval_steps=0.1,
        logging_strategy="steps",
        logging_steps=50,
        save_strategy="steps",
        save_steps=0.1,
        warmup_steps = 5,
        num_train_epochs = 3,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        load_best_model_at_end=True,
        metric_for_best_model="eval_loss",
        greater_is_better=False,
        seed = 42,
        report_to = "none",
    )

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset=dataset_formatted["train"],
    eval_dataset=dataset_formatted["test"],
    dataset_text_field = "text",
    max_seq_length = 2048,
    data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer), #only use when using train_on_responses_only()
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = training_args)

In [ ]:
trainer.train_dataset

In [ ]:
print(tokenizer.decode(trainer.train_dataset['input_ids'][0]))

#### Only Focus on the `Response Part` for the generation

In [ ]:
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<|start_header_id|>user<|end_header_id|>\n\n",
    response_part = "<|start_header_id|>assistant<|end_header_id|>\n\n",
)

In [ ]:
trainer.train_dataset

In [ ]:
# The labels are created which only contain response. Left Padding is implemented and all the padding tokens are given a score of -100 to avoid loss calculation for pad_tokens
trainer.train_dataset['labels'][0]

#### Train the model

In [ ]:
trainer_stats = trainer.train()

#### Saving the model and tokenizer

For full fine-tuning, current technique in Unsloth doesn't work well. Use the below technique to save the model and tokenizer

In [ ]:
# full_model_path = "./llama32-sft-full-counselchat" #use for full finetuning
# trainer.model.save_pretrained(full_model_path)
# trainer.tokenizer.save_pretrained(full_model_path)

Merge the LoRA into the base model and then save 16-bit version (Doesn't work well as of now. Try later with research)

In [ ]:
# peft_merged_model_path = "./llama32-sft-peft-merged-counselchat"
# model.save_pretrained_merged(peft_merged_model_path, tokenizer, save_method = "merged_16bit")

Just save the LoRA Adapters without merging with base model

In [ ]:
peft_model_path = "./llama32-sft-peft-counselchat"

# Or run the two below statements
model.save_pretrained(peft_model_path)
tokenizer.save_pretrained(peft_model_path)

### Inference

In [12]:
# full_model_path = "./llama32-sft-full-counselchat"
peft_model_path = "./llama32-sft-peft-counselchat"

In [ ]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = peft_model_path,
    max_seq_length = 2048,
    load_in_4bit = False, # 4 bit quantization to reduce memory
    load_in_8bit = False, # [NEW!] A bit more accurate, uses 2x memory
    full_finetuning = False, # [NEW!] We have full finetuning now!
    dtype=None, #None for auto-detection. Can be torch.bfloat16 or torch.float16 (will be automatically detected)
    device_map="auto"
)

==((====))==  Unsloth 2025.3.19: Fast Llama patching. Transformers: 4.50.2.
   \\   /|    NVIDIA RTX A6000. Num GPUs = 1. Max memory: 47.413 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.4.1+cu124. CUDA: 8.6. CUDA Toolkit: 12.4. Triton: 3.0.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post1. FA2 = True]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [11]:
FastLanguageModel.for_inference(model)

for idx in range(1,50):

    print(dataset['test']['question'][idx])

    messages = [{"role": "system", "content": "You are an expert mental health professional trained to counsel and guide patients suffering from ill mental-health"},
        {"role": "user", "content": dataset['test']['question'][idx]}]

    prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
        
    inputs = tokenizer(prompt, return_tensors='pt', padding=True, truncation=True).to(model.device)

    outputs = model.generate(**inputs, max_new_tokens=2048, num_return_sequences=1)

    text = tokenizer.decode(outputs[0], skip_special_tokens=True)

    print(text.split("assistant")[1])

    print('---------------------------------------------------')

What makes a healthy marriage last? What makes a good marriage?


What makes a healthy marriage last? What makes a good marriage?I think there are several factors that contribute to a healthy marriage. First and foremost is communication. Being able to communicate effectively is crucial. Being able to communicate effectively means that you are able to listen to each other, express your thoughts and feelings, and be able to resolve conflicts in a healthy way. A healthy marriage also requires a commitment to the relationship and to each other. This means that you are able to put the needs of the relationship first and that you are willing to make sacrifices for the relationship. It also means that you are able to forgive each other and move forward. A healthy marriage also requires a willingness to work through conflicts and challenges together. This means that you are able to be patient and understanding with each other, and that you are able to find solutions that work for both of you.